In [22]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import argparse
import torch

In [23]:
sys.path += ["/Users/khj/Development/pollution/Autoformer"]

In [24]:
def get_y_files():
    paths = glob('./dataset/TRAIN/*.csv')
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

def get_x_files():
    paths = glob('./dataset/TRAIN_AWS/*.csv')
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

In [25]:
x = get_x_files()
x = x.dropna(axis=0)
y = get_y_files()
y = y.dropna(axis=0)
x_plus = y[['연도', '일시', '측정소', 'PM2.5']]
data = pd.merge(x, x_plus, on=['연도', '일시'], how='right')
year_mapping = {0: 2013, 1: 2014, 2: 2015, 3: 2016, 4: 2017, 5: 2018, 6: 2019}
data['일시'] = data['연도'].apply(lambda x: str(year_mapping[x]) + " ") + data['일시']
data['일시'] = pd.to_datetime(data['일시'])
data.drop(['연도'], axis=1, inplace=True)
column_names = ['date', 'point', 'temp', 'degree', 'speed', 'rain', 'humidity', 'station', 'pm']
data.columns = column_names
data = data.dropna(axis=0)
y = y['PM2.5']

In [14]:
data.sort_values(by=['date', 'point'], inplace=True)
data.drop(['point', 'station'], axis=1, inplace=True)
data.to_csv('./dataset/autoformer_test.csv', index=False)


In [15]:
# data.to_csv('./dataset/autoformer_test.csv', index=False)

In [16]:
from Autoformer.data_provider.data_loader import Dataset_Custom

In [17]:
dataset = Dataset_Custom(
    root_path="./dataset",
    data_path="autoformer_test.csv",
    scale=False,
    features="MS",
    target='pm',
)